In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [4]:
from talus_data_analysis.plot import histogram
from talus_data_analysis.elib import Elib
from talus_data_analysis.load import read_excel_from_gdrive
from talus_data_analysis.save import write_df_to_s3
from dotenv import load_dotenv
import tempfile
import sqlite3
import math
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [5]:
load_dotenv()

True

In [6]:
gauth = GoogleAuth(settings_file="../settings.yaml")

In [137]:
ENCYCLOPEDIA_BUCKET = "talus-data-pipeline-encyclopedia-bucket"
DATA_FOLDER = "../data/210511_GryderLab"
S3_FOLDER = "wide/210511_GryderLab"
PROJECT_NAME = "Gryder Contract"

ELIB_FILE = "RESULTS-quant.elib"
peptide_protein_file = "peptide_proteins_results.csv"
peptide_protein_norm_output = "peptide_proteins_normalized.csv"
msstats_groupcompare_output = "msstats_groupcompare.csv"
comparison_matrix_file = "comparison_matrix.csv"

In [138]:
elib_conn = Elib(key=f"{S3_FOLDER}/{ELIB_FILE}", bucket=ENCYCLOPEDIA_BUCKET)

In [139]:
peptide_quants = elib_conn.execute_sql(sql="SELECT * FROM peptidequants;", use_pandas=True)        
peptide_quants = peptide_quants[["PeptideSeq", "TotalIntensity", "SourceFile", "PrecursorCharge"]]
peptide_quants["Run"] = peptide_quants["SourceFile"].apply(lambda x: x.split(".")[0].split("_")[-1])

In [140]:
peptide_to_protein = elib_conn.execute_sql(sql="SELECT * FROM peptidetoprotein;", use_pandas=True)
peptide_to_protein = peptide_to_protein[peptide_to_protein['isDecoy'] == False]
peptide_to_protein = peptide_to_protein[["PeptideSeq", "ProteinAccession"]]

In [141]:
peptide_df = pd.merge(peptide_to_protein, peptide_quants, how="right", on="PeptideSeq")

## Template DF

In [142]:
sample_df = read_excel_from_gdrive(gauth=gauth, key="MS_samples_April2021.xlsx", sheet="Sheet1", remove_unnamed=True)
sample_df = sample_df[(sample_df["Project"] == PROJECT_NAME) & (sample_df["Sample Description"] != "Pooled Sample")]
sample_df["Run"] = sample_df["Run ID"].apply(lambda x: x.split("_")[-1])
sample_df = sample_df.rename(columns={"Sample Name": "BioReplicate", "Sample Description": "Condition"})
sample_df = sample_df[["Run", "BioReplicate", "Condition", "Comparison"]]

In [114]:
# taking out the BG samples
# sample_df = sample_df[~sample_df["Condition"].isin(set(["BG15-n - 24h", "BG15-n - 48h", "BG-15a - 24h", "BG-15a - 48h"]))]

In [143]:
msstats_df = pd.merge(peptide_df, sample_df, how="right", on="Run")

In [144]:
## Add a few required columns and rename header to match MSstats convention
msstats_df = msstats_df.drop(["Run", "Comparison"], axis=1)
msstats_df["IsotopeLabelType"] = "L"
msstats_df["FragmentIon"] = "y0"
msstats_df["ProductCharge"] = "1"
msstats_df = msstats_df.rename(columns={"PeptideSeq": "PeptideSequence",
                                        "ProteinAccession": "ProteinName",
                                        "SourceFile": "Run",
                                        "TotalIntensity": "Intensity"})

In [145]:
msstats_df.head()

,PeptideSequence,ProteinName,Intensity,Run,PrecursorCharge,BioReplicate,Condition,IsotopeLabelType,FragmentIon,ProductCharge
0,YFTQGNCVNLTEALSLYEEQLGR,sp|P52788|SPSY_HUMAN,3616023.500,210511_Talus_Run1.mzML,3,Gryder-1,DMSO - 24h,L,y0,1
1,TFLQTVSSEK,sp|Q96EL3|RM53_HUMAN,174549.125,210511_Talus_Run1.mzML,2,Gryder-1,DMSO - 24h,L,y0,1
2,KIWCFGPDGTGPNILTDITK,sp|P13639|EF2_HUMAN,8197211.500,210511_Talus_Run1.mzML,3,Gryder-1,DMSO - 24h,L,y0,1
3,NVLIFDLGGGTFDVSILTIEDGIFEVK,sp|P11142|HSP7C_HUMAN,9216460.000,210511_Talus_Run1.mzML,3,Gryder-1,DMSO - 24h,L,y0,1
4,NVLIFDLGGGTFDVSILTIEDGIFEVK,sp|P54652|HSP72_HUMAN,9216460.000,210511_Talus_Run1.mzML,3,Gryder-1,DMSO - 24h,L,y0,1


In [146]:
msstats_df.to_csv(f"{DATA_FOLDER}/{peptide_protein_file}")

In [147]:
write_df_to_s3(dataframe=msstats_df, bucket=ENCYCLOPEDIA_BUCKET, key=f"{S3_FOLDER}/{peptide_protein_file.replace('.csv', '.parquet')}", outputformat="parquet")

In [148]:
def get_comparison_matrix(df, filter_target_func=lambda x:x):
    df = df.sort_values(by="Condition")
    dmso_map = {condition: dmso for (condition, dmso) in zip(df["Condition"], df["Comparison"])}
    
    comp_lol = []
    targets = sorted(dmso_map.keys())
    comp_df_index = []
    for i, s in enumerate(targets):
        comp_list = [0 for t in targets]
        if dmso_map[s] in dmso_map:
            # make dmso 0
            comp_list[targets.index(dmso_map[s])] = -1
            # make target itself 1
            comp_list[i] = 1

            comp_lol.append(comp_list)
            comp_df_index.append(f"{filter_target_func(s)}/{dmso_map[s]}")

    comp_df = pd.DataFrame(comp_lol)
    comp_df.index = comp_df_index
    
    return comp_df

In [149]:
comp_matrix = get_comparison_matrix(df=sample_df, filter_target_func=lambda x: x.split(" - ")[0])

In [150]:
comp_matrix

,0,1,2,3,4,5,6,7,8,9,10,11
ARN-505/DMSO - 24h,1,0,0,0,0,0,0,0,-1,0,0,0
ARN-505/DMSO - 48h,0,1,0,0,0,0,0,0,0,-1,0,0
BG-15a/DMSO - 24h,0,0,1,0,0,0,0,0,-1,0,0,0
BG-15a/DMSO - 48h,0,0,0,1,0,0,0,0,0,-1,0,0
BG15-n/DMSO - 24h,0,0,0,0,1,0,0,0,-1,0,0,0
BG15-n/DMSO - 48h,0,0,0,0,0,1,0,0,0,-1,0,0
Biclutimide/DMSO - 24h,0,0,0,0,0,0,1,0,-1,0,0,0
Biclutimide/DMSO - 48h,0,0,0,0,0,0,0,1,0,-1,0,0
R1881/DMSO - 24h,0,0,0,0,0,0,0,0,-1,0,1,0
R1881/DMSO - 48h,0,0,0,0,0,0,0,0,0,-1,0,1


In [151]:
comp_matrix.to_csv(f"{DATA_FOLDER}/{comparison_matrix_file}")

# Run R Script (MSStats) ...

## Write msstats normalized peptide protein df to s3

In [152]:
msstats_df_norm = pd.read_csv(f"{DATA_FOLDER}/{peptide_protein_norm_output}")

In [154]:
msstats_df_norm

,PROTEIN,PEPTIDE,TRANSITION,FEATURE,LABEL,GROUP_ORIGINAL,SUBJECT_ORIGINAL,RUN,GROUP,SUBJECT,INTENSITY,SUBJECT_NESTED,ABUNDANCE,FRACTION,originalRUN,censored
0,sp|A0A1W2PR48|TLE7_HUMAN,LSGLEAPSLQK_2,y0_1,LSGLEAPSLQK_2_y0_1,L,ARN-505 - 24h,Gryder-4,1,1,4,1.000000e+00,1.4,0.000000,1,210511_Talus_Run4.mzML,True
1,sp|A0AVT1|UBA6_HUMAN,FISADVHGIWSR_3,y0_1,FISADVHGIWSR_3_y0_1,L,ARN-505 - 24h,Gryder-4,1,1,4,1.085368e+06,1.4,20.915179,1,210511_Talus_Run4.mzML,False
2,sp|A0AVT1|UBA6_HUMAN,NCFLNLAIPIVVFTETTEVR_3,y0_1,NCFLNLAIPIVVFTETTEVR_3_y0_1,L,ARN-505 - 24h,Gryder-4,1,1,4,3.750285e+05,1.4,19.382067,1,210511_Talus_Run4.mzML,False
3,sp|A0AVT1|UBA6_HUMAN,TVFFESLER_2,y0_1,TVFFESLER_2_y0_1,L,ARN-505 - 24h,Gryder-4,1,1,4,2.064677e+06,1.4,21.842911,1,210511_Talus_Run4.mzML,False
4,sp|A0AVT1|UBA6_HUMAN,YVDLTVSFAPDIDGDEDLPGPPVR_3,y0_1,YVDLTVSFAPDIDGDEDLPGPPVR_3_y0_1,L,ARN-505 - 24h,Gryder-4,1,1,4,1.000000e+00,1.4,0.000000,1,210511_Talus_Run4.mzML,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109843,sp|Q9Y6Q5|AP1M2_HUMAN,VLFELTGR_2,y0_1,VLFELTGR_2_y0_1,L,R1881 - 48h,Gryder-8,12,12,8,5.530902e+05,12.8,17.649025,1,210511_Talus_Run14.mzML,False
109844,sp|Q9Y6X4|F169A_HUMAN,DFGLHMLEDFVDSFTEDALGLR_3,y0_1,DFGLHMLEDFVDSFTEDALGLR_3_y0_1,L,R1881 - 48h,Gryder-8,12,12,8,1.000000e+00,12.8,0.000000,1,210511_Talus_Run14.mzML,True
109845,sp|Q9Y6X4|F169A_HUMAN,GKDFGLHMLEDFVDSFTEDALGLR_3,y0_1,GKDFGLHMLEDFVDSFTEDALGLR_3_y0_1,L,R1881 - 48h,Gryder-8,12,12,8,1.000000e+00,12.8,0.000000,1,210511_Talus_Run14.mzML,True
109846,sp|Q9Y6Y8|S23IP_HUMAN,EVLTLQETLEALSLSEYFSTFEK_3,y0_1,EVLTLQETLEALSLSEYFSTFEK_3_y0_1,L,R1881 - 48h,Gryder-8,12,12,8,5.637411e+04,12.8,14.354615,1,210511_Talus_Run14.mzML,False


In [155]:
write_df_to_s3(dataframe=msstats_df_norm, bucket=ENCYCLOPEDIA_BUCKET, key=f"{S3_FOLDER}/{peptide_protein_norm_output.replace('.csv', '.parquet')}", outputformat="parquet")

## Write msstats groupcompare df to s3

In [156]:
msstats_groupcompare = pd.read_csv(f"{DATA_FOLDER}/{msstats_groupcompare_output}")

In [157]:
msstats_groupcompare

,Protein,Label,log2FC,SE,Tvalue,DF,pvalue,adj.pvalue,issue,MissingPercentage,ImputationPercentage
0,sp|A0A1W2PR48|TLE7_HUMAN,ARN-505/DMSO - 24h,-inf,NaN,NaN,NaN,NaN,0.0,oneConditionMissing,0.500,0.000
1,sp|A0AVT1|UBA6_HUMAN,ARN-505/DMSO - 24h,0.218510,NaN,NaN,0.0,NaN,NaN,NaN,0.250,0.250
2,sp|A0FGR8|ESYT2_HUMAN,ARN-505/DMSO - 24h,-0.457277,NaN,NaN,0.0,NaN,NaN,NaN,0.000,0.000
3,sp|A0MZ66|SHOT1_HUMAN,ARN-505/DMSO - 24h,-0.689042,NaN,NaN,0.0,NaN,NaN,NaN,0.500,0.500
4,sp|A1L0T0|ILVBL_HUMAN,ARN-505/DMSO - 24h,inf,NaN,NaN,NaN,NaN,0.0,oneConditionMissing,0.500,0.000
...,...,...,...,...,...,...,...,...,...,...,...
21385,sp|Q9Y6I4|UBP3_HUMAN,R1881/DMSO - 48h,0.920673,NaN,NaN,0.0,NaN,NaN,NaN,0.000,0.000
21386,sp|Q9Y6M1|IF2B2_HUMAN,R1881/DMSO - 48h,1.344610,NaN,NaN,0.0,NaN,NaN,NaN,0.125,0.125
21387,sp|Q9Y6Q5|AP1M2_HUMAN,R1881/DMSO - 48h,1.468264,NaN,NaN,0.0,NaN,NaN,NaN,0.000,0.000
21388,sp|Q9Y6X4|F169A_HUMAN,R1881/DMSO - 48h,NaN,NaN,NaN,NaN,NaN,NaN,completeMissing,1.000,0.000


In [158]:
write_df_to_s3(dataframe=msstats_groupcompare, bucket=ENCYCLOPEDIA_BUCKET, key=f"{S3_FOLDER}/{msstats_groupcompare_output.replace('.csv', '.parquet')}", outputformat="parquet")